In [10]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json, re
from pathlib import Path
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

In [11]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,A Lenda Super Vascão f.c
1,A Lenda Super Vasco F.c
2,Analove10 ITAQUI GRANDE!!
3,BordonFC
4,BORGES ITAQUI F.C.


In [12]:
# 👉 ajuste o caminho do HTML salvo
HTML_PATH = Path("pagina_liga_serie_C.html")

# carrega html
html = HTML_PATH.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# 1) seletor principal dessa página
nomes = [el.get_text(strip=True) for el in soup.select("p.card-participantes-pontos-corridos__time")]

# 2) fallbacks (caso salve outra variante da página)
if not nomes:
    nomes = [el.get_text(strip=True) for el in soup.select("p.nome-time")]
if not nomes:
    nomes = [(a.get("title") or a.get_text(strip=True)).strip()
             for a in soup.select("a.link-perfil-time") if (a.get("title") or a.get_text(strip=True))]

# 3) fallback final via regex (se o HTML veio “achatado”)
if not nomes:
    for m in re.finditer(r'class="card-participantes-pontos-corridos__time"\s*>\s*([^<]+)<', html, re.I):
        nomes.append(m.group(1).strip())

# dedup + ordena alfabética
visto, times = set(), []
for n in nomes:
    n = re.sub(r"\s+", " ", n).strip()
    if n and n not in visto:
        visto.add(n); times.append(n)
times.sort(key=str.casefold)

print(f"{len(times)} times encontrados")
print(times)

# snippet JS ao lado do HTML (se quiser usar no site)
OUT_JS = HTML_PATH.with_name("participantesLiga_serie_C.js")
OUT_JS.write_text("window.participantesLiga = " + json.dumps(times, ensure_ascii=False, indent=2) + ";\n",
                  encoding="utf-8")
print(f"Snippet salvo em: {OUT_JS}")

20 times encontrados
['cartola scheuer', 'DM Studio', 'Dom Camillo68', 'FBC Colorado', 'FC castelo Branco 2', 'Fedato Futebol Clube', 'Gremiomaniasm', 'HS SPORTS F.C', 'lsauer fc', 'MauHumor F.C.', 'Noah A 10', 'pra sempre imortal fc', 'pura bucha /botafogo', 'PÃO DE QUEIJO FC25', 'seralex', 'Tabajara de Inhaua PB7', 'TATITTA FC', 'Tatols Beants F.C', 'Texas Club 2025', 'TIGRE LEON']
Snippet salvo em: participantesLiga_serie_C.js


In [13]:
JS_PATH = Path("participantesLiga_serie_C.js")  # ajuste o caminho se estiver em outra pasta

# extrai o array do snippet: window.participantesLiga = [ ... ];
m = re.search(r'window\.participantesLiga\s*=\s*(\[[\s\S]*?\])\s*;', JS_PATH.read_text(encoding="utf-8"))
if not m:
    raise ValueError(f"Não consegui achar a lista dentro de {JS_PATH}")

nomes_times = json.loads(m.group(1))

# (opcional) dedup + ordena
nomes_times = sorted(dict.fromkeys(nomes_times), key=str.casefold)

print(len(nomes_times), "times")
nomes_times

20 times


['cartola scheuer',
 'DM Studio',
 'Dom Camillo68',
 'FBC Colorado',
 'FC castelo Branco 2',
 'Fedato Futebol Clube',
 'Gremiomaniasm',
 'HS SPORTS F.C',
 'lsauer fc',
 'MauHumor F.C.',
 'Noah A 10',
 'pra sempre imortal fc',
 'pura bucha /botafogo',
 'PÃO DE QUEIJO FC25',
 'seralex',
 'Tabajara de Inhaua PB7',
 'TATITTA FC',
 'Tatols Beants F.C',
 'Texas Club 2025',
 'TIGRE LEON']

### Buscar IDs dos times no Cartola

In [14]:
# nomes_times = [ 'FBC Colorado', 'Tatols Beants F.C', 'DM Studio', 'Gremiomaniasm', 'Texas Club 2025',
#                 'TIGRE LEON', 'cartola scheuer', 'HS SPORTS F.C', 'Fedato Futebol Clube', 'Dom Camillo68',
#                 'pra sempre imortal fc', 'MauHumor F.C.', 'seralex', 'lsauer fc', 'Tabajara de Inhaua PB7',
#                 'PÃO DE QUEIJO FC25', 'FC castelo Branco 2', 'TATITTA FC', ]

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
from difflib import get_close_matches

def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)

        # 🛡️ Proteção caso a resposta seja uma string (ex: erro HTML ou mensagem)
        if not isinstance(times, list):
            print(f"⚠️ Resposta inesperada para '{nome_time}':", times)
            return None

        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id

    except Exception as e:
        print(f"❌ Erro ao buscar ID para o time '{nome_time}': {e}")
    return None


# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 20
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [15]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_C.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head(30))

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_C.xlsx


,Nome do Time,ID do Time,Link do Time
0,cartola scheuer,3851966,https://cartola.globo.com/#!/time/3851966
1,DM Studio,387186,https://cartola.globo.com/#!/time/387186
2,Dom Camillo68,20696550,https://cartola.globo.com/#!/time/20696550
3,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
4,FC castelo Branco 2,48279389,https://cartola.globo.com/#!/time/48279389
5,Fedato Futebol Clube,18642587,https://cartola.globo.com/#!/time/18642587
6,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
7,HS SPORTS F.C,17887202,https://cartola.globo.com/#!/time/17887202
8,lsauer fc,44810918,https://cartola.globo.com/#!/time/44810918
9,MauHumor F.C.,25751748,https://cartola.globo.com/#!/time/25751748


### Gerar o dicionário ID -> Nome do Time

In [16]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:20])

{3851966: 'cartola scheuer',
 387186: 'DM Studio',
 20696550: 'Dom Camillo68',
 186283: 'FBC Colorado',
 48279389: 'FC castelo Branco 2',
 18642587: 'Fedato Futebol Clube',
 528730: 'Gremiomaniasm',
 17887202: 'HS SPORTS F.C',
 44810918: 'lsauer fc',
 25751748: 'MauHumor F.C.',
 49960687: 'Noah A 10',
 25313333: 'pra sempre imortal fc',
 18661583: 'pura bucha /botafogo',
 47742671: 'PÃO DE QUEIJO FC25',
 29228373: 'seralex',
 47543543: 'Tabajara de Inhaua PB7',
 49180400: 'TATITTA FC',
 212042: 'Tatols Beants F.C',
 1273719: 'Texas Club 2025',
 3424598: 'TIGRE LEON'}

In [17]:
TURNO_INICIO = 20
TOTAL_RODADAS = 19

# Tente usar a exceção específica; senão, caia para Exception
try:
    from cartolafc import CartolaFCError as _CartolaErr
except Exception:
    _CartolaErr = Exception

def campeonato_comecou(api, ids_times):
    """
    Considera que o 2º turno começou se:
      - o mercado indica rodada_atual >= 20 E
      - há pelo menos uma pontuação não-nula na rodada 20 para algum time.
    """
    try:
        rodada_atual = getattr(api.mercado(), "rodada_atual", None)
    except _CartolaErr:
        rodada_atual = None

    if rodada_atual is None or rodada_atual < TURNO_INICIO:
        return False

    for time_id in ids_times.values():
        try:
            p = api.time(time_id=time_id, rodada=TURNO_INICIO).ultima_pontuacao
            if p is not None:
                return True
        except _CartolaErr:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """
    Busca apenas rodadas concluídas: [20, rodada_atual-1].
    """
    pontuacoes = {}
    # fim exclusivo; para parar em rodada_atual-1 use range(..., rodada_atual)
    fim = min(max(rodada_atual, TURNO_INICIO), TOTAL_RODADAS + 1)
    for rodada in range(TURNO_INICIO, fim):
        try:
            time_r = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_r.ultima_pontuacao
        except _CartolaErr as e:
            print(f"⚠️ Erro ao acessar rodada {rodada} (time {time_id}): {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    try:
        rodada_atual = api.mercado().rodada_atual
    except _CartolaErr:
        # fallback seguro se a API falhar
        rodada_atual = TURNO_INICIO

    colunas_turno = [f"Rodada {i}" for i in range(TURNO_INICIO, TOTAL_RODADAS + 1)]

    # Caso o 2º turno ainda não tenha começado (sem pontuações na R20)
    if not campeonato_comecou(api, ids_times):
        print("📌 2º Turno ainda não começou. Criando estrutura com placeholders (0).")
        df = pd.DataFrame(0, index=list(ids_times.keys()), columns=colunas_turno)
        return df

    # Há ao menos uma pontuação na R20 → montar com rodadas concluídas
    dados = {}
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
        # Converte para Series com índice "Rodada N"
        s = pd.Series({f"Rodada {k}": v for k, v in pontuacoes.items()})
        dados[nome] = s

    # DataFrame com as rodadas já buscadas (pode estar vazio se rodada_atual==20)
    df = pd.DataFrame.from_dict(dados, orient="index")

    # Garante TODAS as colunas do 2º turno, preenchendo faltantes com 0
    df = df.reindex(columns=colunas_turno, fill_value=0)

    # Calcula "Lider_Rodada" somente se houver pelo menos uma rodada com algum valor não-nulo
    colunas_com_dados = [c for c in colunas_turno if df[c].notna().any()]
    if len(colunas_com_dados) > 0 and df[colunas_com_dados].astype(float).sum().sum() != 0:
        df.loc['Lider_Rodada'] = df[colunas_com_dados].idxmax(axis=0)
    # Caso contrário, não cria a linha de líder (evita líder "aleatório" com tudo 0)

    return df

# === Exemplo de uso ===
# ids_times = {v: k for k, v in nomes_por_id.items()}  # índice = nome do time
df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

# Visualização como você já faz
display(df_pontuacoes.T)

""


In [19]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

""


### Função para definir a classificação dos times

In [20]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [21]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série C"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head(20))

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,TEAM LOPES 99,TIGRE LEON,13,14,Série C
1,1,2,Fedato Futebol Clube,Dom Camillo68,5,4,Série C
2,1,3,Texas Club 2025,S.E.R. GRILLO,16,10,Série C
3,1,4,HS SPORTS F.C,seralex,6,20,Série C
4,1,5,Noah A 10,Cril Futebol Club,9,2,Série C


ValueError: No objects to concatenate

In [ ]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [ ]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_C.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_C.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_C.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_C.js'

In [ ]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série C"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_C.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_C.js gerado com sucesso!")


✅ Arquivo confrontos_serie_C.js gerado com sucesso!


In [ ]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_C.csv")

display(df_confrontos.head(10))

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,TEAM LOPES 99,TIGRE LEON,13,14
1,1,2,Fedato Futebol Clube,Dom Camillo68,5,4
2,1,3,Texas Club 2025,S.E.R. GRILLO,16,10
3,1,4,HS SPORTS F.C,seralex,6,20
4,1,5,Noah A 10,Cril Futebol Club,9,2
5,1,6,cartola scheuer,SERGRILLO,17,11
6,1,7,Tatols Beants F.C,Super Vasco f.c,15,12
7,1,8,DM Studio,lsauer fc,3,18
8,1,9,pura bucha /botafogo,ITAQUI F. C.,19,7
9,1,10,BORGES CLIMA FUT F.C,KP JUV.,1,8


In [ ]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série C"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série C por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [ ]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_C.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_C.js gerado com sucesso.")


✅ Arquivo resultados_serie_C.js gerado com sucesso.
